In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.5 MB/s eta 0:00:00


In [ ]:
####### Importing Libraries and Packages required #######

import pickle
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

In [ ]:
## Define the ATIS dataset class ##

class ATISDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        encoding = self.tokenizer.encode_plus(
            self.inputs[idx],
            add_special_tokens=True,
            truncation=True,
            padding='max_length',
            max_length=128,
            return_attention_mask=True,
            return_tensors='pt'
        )

        item = {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx])
        }

        return item

In [ ]:
## Load the ATIS dataset ##
data_file = '/content/drive/MyDrive/atis_intents.csv'
data = pd.read_csv(data_file)
data.columns = ['label', 'text']

In [ ]:
## Load the Test Set ##
test_data = pd.read_csv('/content/drive/MyDrive/atis_intents_test.csv')
test_data.columns = ['label', 'text']
test_inputs = list(test_data['text'])
test_labels = list(test_data['label'])
test_labels = [label_map[label] for label in test_labels]
test_dataset = ATISDataset(test_inputs, test_labels)

In [ ]:
## Pre-process the data ##
inputs = list(data['text'])
labels = list(data['label'])
label_map = {label: i for i, label in enumerate(set(labels))}
labels = [label_map[label] for label in labels]

In [ ]:
## Split the data into training and validation sets ##
train_size = int(0.8 * len(data))
train_dataset = ATISDataset(inputs[:train_size], labels[:train_size])
val_dataset = ATISDataset(inputs[train_size:], labels[train_size:])

In [ ]:
## Path to save the files ##
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Training the model ##
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=lambda data: {'input_ids': torch.stack([item['input_ids'] for item in data]),
                                'attention_mask': torch.stack([item['attention_mask'] for item in data]),
                                'labels': torch.stack([item['labels'] for item in data])}
)


trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss
10,3.083500
20,3.048900
30,2.928200
40,2.700400
50,2.414100
60,2.134500
70,1.841300
80,1.650000
90,1.366900
100,1.398900


Step,Training Loss
10,3.083500
20,3.048900
30,2.928200
40,2.700400
50,2.414100
60,2.134500
70,1.841300
80,1.650000
90,1.366900
100,1.398900


TrainOutput(global_step=249, training_loss=1.2848310499306184, metrics={'train_runtime': 4982.2712, 'train_samples_per_second': 0.799, 'train_steps_per_second': 0.05, 'total_flos': 261908300782080.0, 'train_loss': 1.2848310499306184, 'epoch': 1.0})

In [ ]:
## Evaluating the results ##
eval_results = trainer.evaluate(eval_dataset=val_dataset)


In [ ]:
print(eval_results)

{'eval_loss': 0.36867642402648926, 'eval_runtime': 383.5991, 'eval_samples_per_second': 2.596, 'eval_steps_per_second': 0.042, 'epoch': 1.0}


In [ ]:
## Storing the pickle file for the model ##

model_data = {'model': model}

## Open a file for writing binary data
with open('model.pickle', 'wb') as f:
    ## Dump the model data to the file
    pickle.dump(model_data, f)

In [ ]:
## Model on the test set ##
predictions = trainer.predict(test_dataset)
predicted_labels = [np.argmax(pred) for pred in predictions.predictions]

In [ ]:
## Evaluate the Model on the test set ##
accuracy = accuracy_score(test_labels, predicted_labels)
print('Test accuracy:', accuracy)

Test accuracy: 0.9712140175219024


In [ ]:
################################################################################################################################################################################################